## 1. Данные (отзывы + извлечение названий из метадаты)

**Я использовала 5-core датасет "Musical instruments" и полный metadata-датасет для данной категории. Все промежуточные файлы можно найти [здесь](https://kaggle.com/datasets/b5797b8550189163ceb2cce1f1527c2241e8bdffaa3dd7b7b336dd69ab1c9fef)**

In [24]:
from textblob import TextBlob

In [106]:
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [25]:
from nltk.corpus import stopwords
sw = stopwords.words('english')

In [7]:
# import RAKE
from tqdm import tqdm
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
import re

In [1]:
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Musical_Instruments_5 (1).json.gz')

In [2]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"
...,...,...,...,...,...,...,...,...,...
10256,A14B2YH83ZXMPP,B00JBIVXGC,Lonnie M. Adams,"[0, 0]","Great, just as expected. Thank to all.",5.0,Five Stars,1405814400,"07 20, 2014"
10257,A1RPTVW5VEOSI,B00JBIVXGC,Michael J. Edelman,"[0, 0]",I've been thinking about trying the Nanoweb st...,5.0,"Long life, and for some players, a good econom...",1404259200,"07 2, 2014"
10258,AWCJ12KBO5VII,B00JBIVXGC,Michael L. Knapp,"[0, 0]",I have tried coated strings in the past ( incl...,4.0,Good for coated.,1405987200,"07 22, 2014"
10259,A2Z7S8B5U4PAKJ,B00JBIVXGC,"Rick Langdon ""Scriptor""","[0, 0]","Well, MADE by Elixir and DEVELOPED with Taylor...",4.0,Taylor Made,1404172800,"07 1, 2014"


In [3]:
meta_df = getDF('meta_Musical_Instruments.json.gz')

In [5]:
meta_df

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Musical Instruments, Drums & Percussion, Hand...",,[Cricket Rubbing the spine with the wooden sti...,,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,"[B00NP8GYVS, B00NP80XMO, B00NP8M098]",,WADSUWAN SHOP,"[Wood percussion, Owl whistle*, Includes woode...","[>#141,729 in Musical Instruments (See Top 100...",[],Musical Instruments,,"December 2, 2013",,0000989983,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Musical Instruments, Drums & Percussion, Hand...",,[Frog - Rubbing its spine with the wooden stic...,,"Wooden Percussion 3 Piece Set Frog, Cricket an...","[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ...",,WADSUWAN SHOP,"[Wood percussion, Small 3 inches, Creates orig...","[>#1,622 in Musical Instruments (See Top 100 i...",[],Musical Instruments,,"December 2, 2013",$0.91,0000098906,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Musical Instruments, Instrument Accessories, ...",,[Vivaldi's famous set of four violin concertos...,,Hal Leonard Vivaldi Four Seasons for Piano (Or...,[],,Hal Leonard,"[., ., .]","[>#330,653 in Musical Instruments (See Top 100...",[],Musical Instruments,,"May 10, 2011",$62.93,0041291905,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,[],,"[The Turn of the Screw (op. 54) vocal score, p...",,The Turn of the Screw (vocal score),"[0486266842, 0793507669, 0393008789, 142341280...",,Boosey &amp; Hawkes,[],"[>#86,354 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"May 23, 2007",$107.79,0060015500,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,[],,[],,Suite for Organ (including the Trumpet Volunta...,[],,,[],"[>#482,025 in Musical Instruments (See Top 100...",[],Musical Instruments,,"February 8, 2013",,0193757710,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120305,"[Musical Instruments, Amplifiers & Effects, Gu...",,[This packages includes:<br> <br> 10 pcs 3pdt ...,,10 pcs 3pdt Stomp Footswitch incl PCB incl met...,[],,3pdt,[5 pcs 9 Pole Footswitch for True Bypass Switc...,"[>#23,334 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"June 25, 2016",$7.99,B01HJDOF2Y,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
120306,"[Musical Instruments, Amplifiers & Effects, Gu...",,[This packages includes:<br> <br> 5 pcs 3pdt T...,,"5 pcs 3pdt Stomp Footswitch incl. PCB, metal w...","[B015334J9G, B01L6GU858, B003HLW14W, B012CF181...",,3pdt,[5 pcs 9 Pole Footswitch for True Bypass Switc...,"[>#40,026 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"June 25, 2016",$0.91,B01HJDJ1PA,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
120307,"[Musical Instruments, Drums & Percussion, Hand...",,[These high quality clear crystal singing bowl...,,Optical Clear Quartz Crystal Singing Bowl Note...,[],,Crystal Energy Bowls,[High quality clear crystal singing bowl is wi...,"[>#481,984 in Musical Instruments (See Top 100...",[],Musical Instruments,,"June 25, 2016",,B01HJEFFTK,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
120308,"[Musical Instruments, Drums & Percussion, Hand...",,[Frog - Rubbing its spine with the wooden stic...,,2x Deluxe Small 2&quot; Wood Frog Guiro Rasp -...,[],,RA,"[Wood percussion, Small 2 inches, Packs of 2 P...","[>#76,917 in Musical Instruments (See Top 100 ...","[B00NP80XMO, B00NP8GYVS, 0000098906, B0149IWDS...",Musical Instruments,"class=""a-bordered a-horizontal-strip

In [6]:
asins = df['asin'].unique().tolist()
f_meta = meta_df[meta_df['asin'].isin(asins)]

In [21]:
len(f_meta) # из 900 товаров названия есть для 791

791

In [19]:
ff = f_meta[['title', 'asin']]

In [20]:
ff

,title,asin
190,Monster Standard 100-21' Instrument Cable - St...,B00004Y2UT
234,Yamaha FC5 Compact Sustain Pedal for Portable ...,B00005ML71
286,Hosa MID-305BK 5-Pin DIN to 5-Pin DIN MIDI Cab...,B000068NTU
288,Fender 18 Feet California Clear Instrument Cab...,B000068NSX
304,Hosa XVS-101F XLR3F to Right Angle 3.5 mm TRS ...,B000068NZG
...,...,...
79909,Snark S-1 Son of Snark Guitar and Bass Tuner,B00FXKIG5I
81520,"Planet Waves NS Artist Guitar Capo, Black",B00GTSM8FW
81954,Apogee JAM 96k Guitar Interface Input for iPad...,B00H02C9TG
86060,Elixir Strings 80/20 Bronze Acoustic Guitar St...,B00J4TBMVO


In [55]:
ff.to_csv('title2asin.csv', index=False)

In [58]:
f_df = df[['reviewText', 'asin']]
f_df.to_csv('text2asin.csv', index=False)

## 2. Cпособы извлечь названия продуктов из отзывов:
1. Извлечь коллокации любым способом (например, textrank), а потом векторизовать их бертом (или не бертом) и отсортировать по семантической близости к названию категории или продукта, чтобы исключить обычные языковые коллокации
2. Опираясь на названия, составить словарь сущностей, извлечь содержащие его нграммы
3. Обучить bert-based NER (подавать, например, название продукта + какой-нибудь кастомный сепаратор + текст отзыва)

- Для первого способа надо, как минимум, знать категорию, а как максимум, название продукта для каждого отзыва 
- Для второго тоже нужно знать название продукта и он годится только если список наименований не очень большой
- Для третьего еще нужны обучающие данные + он дорогой с точки зрения вычислений

## 3. Реализация одного из способов
(в данном случае первого, потому что названия известны, но список наименований большой)

In [4]:
txt2a = pd.read_csv('/kaggle/input/amazon-reviews/text2asin.csv')
ttl2a = pd.read_csv('/kaggle/input/amazon-reviews/title2asin.csv')

In [63]:
def text_append(asin): # объединим тексты отзывов одного продукта
    global txt2a
    
    txts = txt2a[txt2a['asin']==asin]
    text = ' '.join([str(i) for i in txts['reviewText'].tolist()])
    
    return text

In [64]:
ttl2a['texts'] = ttl2a['asin'].apply(text_append)
ttl2a.to_csv('ttl2a_text.csv', index = False)

In [ ]:
rake = RAKE.Rake(sw)

In [99]:
asin2kw = {}
asins = ttl2a['asin'].tolist()
for idx, i in tqdm(enumerate(ttl2a['texts'].tolist())): # извлечение ключевых слов
    rake_kw = rake.run(i, maxWords=3, minFrequency=1)
    rake_kw = [j[0] for j in rake_kw] 
    
    blob = TextBlob(i) # оставим только именные группы
    np = blob.noun_phrases
    kw = set(np).intersection(rake_kw)
    
    asin2kw[asins[idx]] = list(kw)
    

791it [00:43, 18.18it/s]


In [101]:
with open('asin2kw.json', 'w') as fp: # бегаю между юпитером и каглом, поэтому приходится все сохранять и выгружать обратно
    json.dump(asin2kw, fp)

In [2]:
with open('/kaggle/input/amazon-reviews/asin2kw.json', 'r') as fp:
    asin2kw = json.load(fp)

In [16]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased").to('cuda')

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [37]:
def return_5_closest(kw, title): # векторизует название и ключевые слова, возвращает 5 наиболее похожих на название
    
    ttl_and_kw = [title] + kw
    encoded_input = tokenizer(ttl_and_kw, padding=True, truncation=True, max_length=24, return_tensors='pt').to('cuda')
    
    with torch.no_grad():
        model_output = model(**encoded_input)

    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    q_vec = sentence_embeddings[0:1].cpu().detach().numpy()
    corpus = sentence_embeddings[1:].cpu().detach().numpy()
    
    if len(q_vec) == 0 or len(corpus) == 0:
        return []
    else:
        res = cosine_similarity(corpus, q_vec).reshape(1, -1)[0]
        if len(res) > 5:
            n=5
        else:
            n = len(res)
            
        idx = np.argpartition(res, -n)[-n:]
        idx = sorted(idx, key=lambda x: res[x])
        otp = [kw[i] for i in idx]
        return otp
    

In [38]:
top5_asin2kw = {}

for i in tqdm(ttl2a['asin'].tolist()):
    title = ttl2a[ttl2a['asin']==i]['title'].tolist()[0]
    kw = asin2kw[i]
    
    top5_asin2kw[i] = return_5_closest(kw, title)
    

100%|██████████| 791/791 [00:36<00:00, 21.52it/s]


In [40]:
with open('top5_asin2kw.json', 'w') as fp: 
    json.dump(top5_asin2kw, fp)

In [3]:
with open('/kaggle/input/amazon-reviews/top5_asin2kw.json', 'r') as fp:
    top5_asin2kw = json.load(fp)

In [13]:
ttl2a_text = pd.read_csv('/kaggle/input/amazon-reviews/ttl2a_text.csv')

In [14]:
ttl2a_text

,title,asin,texts
0,Monster Standard 100-21' Instrument Cable - St...,B00004Y2UT,So good that I bought another one. Love the h...
1,Yamaha FC5 Compact Sustain Pedal for Portable ...,B00005ML71,I got it to have it if I needed it. I have fou...
2,Hosa MID-305BK 5-Pin DIN to 5-Pin DIN MIDI Cab...,B000068NTU,Got this cable to run a rockband keyboard cont...
3,Fender 18 Feet California Clear Instrument Cab...,B000068NSX,This Fender cable is the perfect length for me...
4,Hosa XVS-101F XLR3F to Right Angle 3.5 mm TRS ...,B000068NZG,The listing for this item can be confusing if ...
...,...,...,...
786,Snark S-1 Son of Snark Guitar and Bass Tuner,B00FXKIG5I,I seem to have problem with getting it to &#34...
787,"Planet Waves NS Artist Guitar Capo, Black",B00GTSM8FW,"This works well. Has nice ergonomics, fits al..."
788,Apogee JAM 96k Guitar Interface Input for iPad...,B00H02C9TG,Notice in the name guitar INPUT. This is not t...
789,Elixir Strings 80/20 Bronze Acoustic Guitar St...,B00J4TBMVO,Just put these on my Martin DCX1E and they sou...


In [19]:
asin2ngrams = {} # не стала удалять стоп-слова, чтобы в нграммы не попадали какие-то случайные знаменательные слова
pattern = r'\b[A-Za-z]+\b'
for i in top5_asin2kw:
    asin2ngrams[i] = []
    text = ttl2a_text[ttl2a_text['asin']==i]['texts'].tolist()[0]
    for w in top5_asin2kw[i]:
        left = re.findall(pattern + ' ' + w, text)
        right = re.findall(w + ' ' + pattern, text)
        lst = left + right
        asin2ngrams[i].extend(lst)
        
    

## 4. Ранжирование

In [24]:
joined_text = '\n'.join(ttl2a_text['texts'].tolist())

In [37]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
fourgram_measures = nltk.collocations.QuadgramAssocMeasures()

In [92]:
finder2 = BigramCollocationFinder.from_words(joined_text.split())
finder3 = TrigramCollocationFinder.from_words(joined_text.split())
finder4 = QuadgramCollocationFinder.from_words(joined_text.split())

finder2.apply_freq_filter(5)
finder3.apply_freq_filter(5)
finder4.apply_freq_filter(5)

In [54]:
all_ngrams = []
for i in asin2ngrams:
    all_ngrams.extend(asin2ngrams[i])
    
bi = [i for i in all_ngrams if len(i.split())==2]
tri = [i for i in all_ngrams if len(i.split())==3]
quad = [i for i in all_ngrams if len(i.split())==4]

In [71]:
def get_score(kw_list, func, finder):
    res = []
    for i in kw_list:
        score = finder.score_ngram(func, *i.split())
        if not score:
            score = -10**9
        el = (i, score)
        res.append(el)
    return res

In [93]:
b_t = get_score(bi, bigram_measures.student_t, finder2)
b_x = get_score(bi, bigram_measures.chi_sq, finder2)
b_p = get_score(bi, bigram_measures.pmi, finder2)

t_t = get_score(tri, trigram_measures.student_t, finder3)
t_x = get_score(tri, trigram_measures.chi_sq, finder3)
t_p = get_score(tri, trigram_measures.pmi, finder3)

q_t = get_score(quad, fourgram_measures.student_t, finder4)
q_x = get_score(quad, fourgram_measures.chi_sq, finder4)
q_p = get_score(quad, fourgram_measures.pmi, finder4)

In [94]:
t = b_t + t_t + q_t
x = b_x + t_x + q_x
p = b_p + t_p + q_p

In [95]:
t = sorted(set(t), key=lambda x: x[1], reverse=True)
x = sorted(set(x), key=lambda x: x[1], reverse=True)
p = sorted(set(p), key=lambda x: x[1], reverse=True)

In [96]:
t[:30] #student_t

[('the guitar', 19.323414511847968),
 ('a nice', 18.37720997005047),
 ('my guitar', 18.043179500802484),
 ('the strings', 15.4709113586912),
 ('these strings', 13.885395277830444),
 ('strings are', 12.789598810338175),
 ('acoustic guitar', 11.879331296800414),
 ('guitar and', 11.223112526931542),
 ('strings on', 10.506075726614567),
 ('this guitar', 9.907724565083972),
 ('a guitar', 9.768062940133566),
 ('very nice', 9.32249152556826),
 ('These strings', 8.576735337907289),
 ('strings for', 8.005996869484134),
 ('guitar for', 7.287310969080698),
 ('playing guitar', 7.2162149533987625),
 ('this tuner', 7.137658550760066),
 ('change strings', 7.094491296444779),
 ('tuner is', 6.914837460876533),
 ('the tuner', 6.782683592960074),
 ('my pedal board', 6.707339410397258),
 ('AC adapter', 5.653396484847251),
 ('an acoustic guitar', 5.565848581857782),
 ('the sound quality', 5.452025608054476),
 ('guitar strings', 5.292025109512348),
 ('sound quality is', 5.282067412841273),
 ('This guitar', 

In [97]:
x[:30] #chi_square

[('my local music store', 777278570.7991624),
 ('have hard plastic cases', 141202948.27186087),
 ('a cigar box guitar', 120405730.13660802),
 ('my local guitar shop', 57869572.75635922),
 ('planet waves cables', 45693081.48669101),
 ('hard plastic cases and', 36857166.989413574),
 ('lifetime warranty is', 861629.1987178145),
 ('the snare wires', 753838.802137136),
 ('more expensive units', 456942.0272512103),
 ('big fan of', 432862.5703886085),
 ('my pedal board', 355714.1607994526),
 ('for classic rock', 262607.0111997047),
 ('a gig bag', 236305.1907851208),
 ('gig bag for', 188213.7380559106),
 ('a big fan', 186151.18110759484),
 ('right amount of', 135875.00609425912),
 ('strap locks on', 121259.42237199446),
 ('these strap locks', 115729.55920563744),
 ('a microfiber cloth', 114764.9905258351),
 ('the headphone jack', 109910.98806393177),
 ('this gig bag', 109014.17249286028),
 ('an acoustic guitar', 96505.76174979168),
 ('a headphone jack', 95232.104311921),
 ('a 9v battery', 9225

In [98]:
p[:30] #pmi

[('my local music store', 26.497880835366963),
 ('have hard plastic cases', 24.750825567149526),
 ('a cigar box guitar', 24.25352141035036),
 ('my local guitar shop', 23.4632740268094),
 ('hard plastic cases and', 22.811785097874733),
 ('planet waves cables', 22.43496504722131),
 ('lifetime warranty is', 17.28972709444671),
 ('the snare wires', 16.938856855414173),
 ('more expensive units', 15.913641641574483),
 ('big fan of', 14.909600306548406),
 ('for classic rock', 14.779334556062196),
 ('a microfiber cloth', 14.424311677461432),
 ('these strap locks', 14.314606280106279),
 ('a 9v battery', 13.812569845624683),
 ('a big fan', 13.79423329844374),
 ('a headphone jack', 13.702972318444633),
 ('the headphone jack', 13.585624640608795),
 ('strap locks on', 13.58418692475081),
 ('different gauges and', 13.48203884426476),
 ('gig bag for', 13.459689553919386),
 ('strap buttons on', 13.338226032275955),
 ('a gig bag', 13.190285718697787),
 ('right amount of', 12.984586714427078),
 ('my ped

На мой человеческий взгляд, лучше всего справился pmi. Хи-квадрат показывает похожие результаты, а стъюдент выводит в топ биграммы с частотными словами.


## 5. Вывод результатов для 5 продуктов

In [103]:
#ttl2a_text
import random
idxs = random.sample([i for i in range(len(ttl2a_text))], 5)
ttls = [ttl2a_text['title'].tolist()[i] for i in idxs]
asins = [ttl2a_text['asin'].tolist()[i] for i in idxs]

for i in range(5):
    print(ttls[i])
    print('-'*15)
    for j in asin2ngrams[asins[i]]:
        print(j)
    print('='*15)
    print('\n')


Fender Frontman 25R Electric Guitar Amplifier
---------------
the bass knob
bass knob just
a well-made tube amp
well-made tube amp or
of digital amps
digital amps and
your electric guitar
my electric guitar
electric guitar sound


Gibraltar SC-DC Hi Hat Drop Clutch
---------------
during double pedal
you double pedal
double pedal licks
double pedal a
a drop clutch
this drop clutch
This drop clutch
drop clutch its
drop clutch because
drop clutch is
a standard clutch
and stickwork chokes
stickwork chokes the
your hi-hat pedal


MXR M102 Dyna Comp Compresor Pedal
---------------
a distortion pedal
broken distortion pedal
distortion pedal and
distortion pedal so
for compressor pedal
compressor pedal out
this compression pedal
Boss compression pedal
compression pedal to
compression pedal for
The dyna comp
dyna comp does


Behringer Super Fuzz SF300 3-Mode Fuzz Distortion Instrument Effects Pedal
---------------
a fuzz
a fuzz
a fuzz
of fuzz
two fuzz
This fuzz
this fuzz
This fuzz
necessarily 

**Про объединение синонимов: мне не понадобилось, потому что я изначально знала название продукта. Но если вдруг такой разметки нет, можно попробовать кластеризацию, замеря расстояние любым доступным способом от левенштейна до косинусного расстояния между векторами из очень умного энкодера. Правда, здесь возникает проблема: если автор в одном отзыве сравнивает свои старые и новые часы, а потом говорит "хорошие часы", то чтобы понять, к каким часам это относится, нам нужна какая-то модель, которая хорошо умеет в разрешение анафоры, и полный контекст (звучит как очень дорогой по вычислениям вариант)**